In [ ]:
import tkinter as tk
from tkinter import filedialog
from tkinter import simpledialog
from pypdf import PdfReader, PdfWriter
import pdfplumber
import pandas as pd
import os
import win32com.client as win32

# 彈出視窗讓使用者選擇檔案
root = tk.Tk()
root.withdraw()  # 隱藏主視窗

# 讓使用者選擇 PDF 檔案
pdf_path = filedialog.askopenfilename(title="選擇 PDF 檔案", filetypes=[("PDF Files", "*.pdf")])

if not pdf_path:
    print("未選擇檔案，程式結束。")
    exit()

# 讓使用者輸入密碼
pdf_password = simpledialog.askstring("輸入密碼", "請輸入 PDF 密碼：", show="*")

if not pdf_password:
    print("未輸入密碼，程式結束。")
    exit()

# 解密 PDF 並儲存成新的無密碼 PDF
unlocked_pdf = "unlocked.pdf"

try:
    # 解密 PDF 並儲存成新的無密碼 PDF
    reader = PdfReader(pdf_path)
    reader.decrypt(pdf_password)

    writer = PdfWriter()
    for page in reader.pages:
        writer.add_page(page)

    with open(unlocked_pdf, "wb") as f:
        writer.write(f)

    # === 步驟 2：用 pdfplumber 讀取表格 ===
    all_tables = []

    with pdfplumber.open(unlocked_pdf) as pdf:
        for i, page in enumerate(pdf.pages):
            tables = page.extract_tables()
            for table in tables:
                if not table:
                    continue
                df = pd.DataFrame(table[1:], columns=table[0])
                df["頁碼"] = i + 1  # 加上頁碼欄位（可省略）
                all_tables.append(df)

    # 合併所有表格並儲存為 Excel
    final_df = pd.concat(all_tables, ignore_index=True)

    # 使用 PDF 檔名作為 Excel 輸出名稱（不含副檔名）
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]
    
    # 設定儲存的資料夾路徑
    save_folder = os.path.join(os.getcwd(), 'datasets')  # 使用 os.getcwd() 確保當前工作目錄
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)  # 如果資料夾不存在，則建立

    # 儲存的 Excel 檔案路徑
    excel_output = os.path.join(save_folder, f"{base_name}.xlsx")

    # 使用 pandas 的 ExcelWriter 輸出為 Excel
    with pd.ExcelWriter(excel_output, engine='xlsxwriter') as writer:
        final_df.to_excel(writer, index=False, sheet_name='Sheet1')

    # 使用 pywin32 來加密 Excel
    excel = win32.Dispatch('Excel.Application')
    workbook = excel.Workbooks.Open(os.path.abspath(excel_output))
    workbook.Password = pdf_password  # 設置密碼
    workbook.SaveAs(os.path.abspath(excel_output))  # 儲存並保護密碼
    workbook.Close()
    excel.Quit()

    # 清除中間檔（可選）
    os.remove(unlocked_pdf)

    print(f"✅ 已成功轉換為加密的 Excel：{excel_output}")

except Exception as e:
    print(f"發生錯誤: {e}")


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


發生錯誤: name '__file__' is not defined
